# KR260 HLS QPSK Demod with GNURadio

## Setup

### Mounting Development Machine
This notebook utilizes `sshfs` to mount the development directory on the KR260 board to avoid transferring files bewteen machines.  

The development machine that was used to build the KR260 FPGA image is named `dev-wks`.  To mount the development directory on `dev-wks`, first will need to create a folder to mount the directory, then execute the following from the Jupyter Terminal:

```bash
cd /home/root/jupyter_notebooks
mkdir puch
sshfs sdr@dev-wks:/home/sdr/workspace/puch-workspace/HLS-QPSK-Demod-Baseband_002 /home/root/jupyter_notebooks/puch
```

If `sshfs` is not installed, then execute `sudo apt install sshfs` on the Jupyter Terminal.

### Unmounting Development Machine
To unmount the directory
` fusermount -u /home/root/jupyter_notebooks/dev-wks/`


### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

# Initilize

#### Bokeh Plot Function

In [1]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
import numpy as np
import math
import zmq
import sys
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import threading

def plot_time(in_signal,out_signal=None):
    t = np.linspace(0,len(in_signal),len(in_signal))
    output_notebook()
    p = figure(title = "Input & Output Signal")
    
    if out_signal is not None:
        p.line(t,out_signal,legend_label="Output Signal",line_color="red",line_width=1)
    p.line(t,in_signal,legend_label="Input Signal",line_color="blue",line_width=3)
    show(p)

### Create a PUSH Socket


In [ ]:
_PROTOCOL = "tcp://"
_SERVER = "192.168.1.233"          # PYNQ Board
_PUSH_PORT = ":20000"
_PUSH_ADDR = _PROTOCOL + _SERVER + _PUSH_PORT
push_context = zmq.Context()
push_sock = push_context.socket (zmq.PUSH)
rc = push_sock.bind (_PUSH_ADDR)


### Create a PULL Socket

In [2]:
_PROTOCOL = "tcp://"
_SERVER = "192.168.1.13"          # Host Computer
_PULL_PORT = ":10000"
_PULL_ADDR = _PROTOCOL + _SERVER + _PULL_PORT
pull_context = zmq.Context()
pull_sock = pull_context.socket (zmq.PULL)
pull_sock.setsockopt(zmq.RCVBUF, 1024*1024)
rc = pull_sock.connect (_PULL_ADDR)

#### Load Overlay

In [3]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time

#import Python library from repo:
import sys
sys.path.append('/home/root/jupyter_notebooks/puch/')
import fpga.py.puch as puch
import fpga.lib.timestamp.sw.timestamp_regmap as timestamp_regmap
import fpga.lib.led_reg.sw.led_regmap as led_regmap
import fpga.lib.HLS_QPSK_Demod.sw.qpsk_regmap as qpsk_regmap


# Load the overlay
overlay = Overlay('../../overlays/KR260_HLS_QPSK_Demod/output/kr260_hls_qpsk_demod.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0
led_module   = overlay.led_reg_0
timestamp    = overlay.Timestamp_0
qpsk_demod   = overlay.QPSK_Demod_Top_0

# Configure QPSK Syncword, and reset sync lock
qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_WORD_ADDR,0xDEADBEEF)
qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_RESET_ADDR,1)

##### Print out the register map for DMA and HLS Gain Blocks

In [ ]:
overlay.ip_dict

##### Display Time Stamp Register
The time stamp is burned into the FPGA during the build process

In [4]:
print("FPGA Build Timestamp:  " + puch.get_timestamp_str(timestamp))

FPGA Build Timestamp:  2025/3/19 9:46:56


##### Toggle USER_LED[1:0] on/off

In [ ]:
for i in range(10):
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x1)
    time.sleep(0.25)
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x2)
    time.sleep(0.25)
led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x0)

### Configure the QPSK Demod Control Register

Want to enable auto restart, so the FPGA block will always be read to receive samples


In [5]:
fin_tBits, fin_fBits = puch.get_format(qpsk_demod,qpsk_regmap.RegMap.F_IN_ADDR)
print("F_IN : (" + str(fin_tBits) + ", " + str(fin_fBits) + ")")
fout_tBits, fout_fBits = puch.get_format(qpsk_demod,qpsk_regmap.RegMap.F_OUT_ADDR)
print("F_IN : (" + str(fout_tBits) + ", " + str(fout_fBits) + ")")

F_IN : (16, 12)
F_IN : (16, 12)


### Read I/Q from ZMQ Socket from GNURadio

In [18]:
data = pull_sock.recv()
data = np.frombuffer(data, dtype=np.complex64, count=1024*2) # Read 128 Symbols
data = np.array(data)
data = data.astype('complex64')*(2**fin_fBits)
xi = data.real
xq = data.imag
xi = xi.astype(np.int32)
xq = xq.astype(np.int32)
xiq = (xi<<16) + xq
plot_time(xi)

Loading BokehJS ...

### DMA Transfer the Sample buffer from ARM Processor to the HLS QPSK Demodulator

In [14]:
# 16 Samples Per Symbol, therefore create read buffer /16
SamplesPerSym = 16
NumSamples = len(xiq)
NumSymbols = int(NumSamples/SamplesPerSym/4)
print("NumSamples = " + str(NumSamples))
print("NumSymboles = " + str(NumSymbols))

# Clear the buffer write address
qpsk_demod.mmio.write(qpsk_regmap.RegMap.WR_RAM_ADDR_CTRL_ADDR,1)

# Configure Sync Word
qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_RESET_ADDR,1)
qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_WORD_ADDR,0xDEADBEEF)

# Allocate buffers for the input and output signals
in_buffer = allocate(shape=(NumSamples,), dtype=np.int32)

# Copy the samples to the in_buffer
np.copyto(in_buffer,xiq)

# Start HLS QPSK Demodulator
qpsk_demod.mmio.write(qpsk_regmap.RegMap.AP_CONTROL_ADDR,0)
qpsk_demod.mmio.write(qpsk_regmap.RegMap.AP_CONTROL_ADDR,1)

# Trigger the DMA transfer and wait for the result
import time
start_time = time.time()
dma.sendchannel.transfer(in_buffer)
print("Done tranfer in_buffer")
dma.sendchannel.wait()
print("Done send channel wait")

stop_time = time.time()
hw_exec_time = stop_time-start_time

print('DMA Transfer Execution Time  :',hw_exec_time,' sec')

# Plot to the notebook
#plot_time(t,samples,2000,out_signal=out_buffer)

# Free the buffers
in_buffer.close()

print("Sync Lock Indecator  : " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.SYNC_LOCK_ADDR)))
print("Symbols in the Buffer: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)))


NumSamples = 2048
NumSymboles = 32
Done tranfer in_buffer
Done send channel wait
DMA Transfer Execution Time  : 0.0028269290924072266  sec
Sync Lock Indecator  : 1
Symbols in the Buffer: 102


### Read QPSK Demodulator Output Buffer

In [15]:
rd_len = int(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)/SamplesPerSym)
print("Number of DWORDs to Read: " + str(rd_len))
dout = []
for i in range(rd_len):
    qpsk_demod.mmio.write(qpsk_regmap.RegMap.RD_RAM_ADDR_ADDR,i)
    dout.append(qpsk_demod.mmio.read(qpsk_regmap.RegMap.RD_RAM_DATA_ADDR))
    
for i in dout:
    print(str(hex(i)))

Number of DWORDs to Read: 6
0xdeadbeef
0xdeadbeef
0xdeadbeef
0xdeadbeef
0xdeadbeef
0xdeadbeef


#### Clear QPSK Demodulator Output Buffer Pointer

In [ ]:
print("WR Capture Address: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)))
print("RD Capture Address: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.RD_RAM_ADDR_ADDR)))
qpsk_demod.mmio.write(qpsk_regmap.RegMap.WR_RAM_ADDR_CTRL_ADDR,1)
print("WR Capture Address: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)))
print("SYNC Lock         : " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.SYNC_LOCK_ADDR)))
qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_RESET_ADDR,1)
print("SYNC Lock         : " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.SYNC_LOCK_ADDR)))